In [4]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [5]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["round summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Offer your assitance and then introduce yourself casually
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 2. TURN: 1 


As you approach the young lady, you decide to lighten the mood with a touch of humor. With a confident smile, you say, "Did it hurt when you fell from heaven? Because it looks like you've just made a crash landing in my heart."

Mitsuha looks up at you, a bit surprised by the sudden appearance of a stranger but then giggles at the cheesy line. "That's one of the oldest ones in the book, but I'll give you points for trying to make me laugh while I'm sitting here feeling quite embarrassed about my little mishap," she responds with a cheeky smile, her big brown eyes twinkling with amusement. "I'm Mitsuha Tomoe. And you are?"

possible actions:
1. Tell her your name and ask if she needs help with her injury.
2. Joke about being an undercover angel sent to rescue her.
3. Ask her what happened and how she ended up getting injured.

I select option 2. TURN: 2 


With a playful grin, you lean in slightly and lower your voice as if sharing a secret, "Actually, I'm an undercover angel sent to rescue you. It seems my timing was impeccable."

Mitsuha laughs, the sound light and genuine, "An undercover angel, huh? That's a new one. Well, Mr. Angel, it seems I could use a bit of rescuing. I twisted my ankle when I slipped on that patch of wet leaves over there," she says, pointing to a spot on the path. "I guess I was too caught up in the beauty of this place to watch where I was stepping."

She attempts to stand up, wincing slightly as she puts weight on her injured ankle. 

possible actions:
1. Offer to help her stand and suggest walking her back to the hotel for proper care.
2. Insist on carrying her to make sure she doesn't strain her ankle further.
3. Ask if she has anyone she can call for help.

I select option 2. TURN: 3 


Seeing Mitsuha in discomfort, you step closer and offer, "No need to strain yourself further. Allow me to carry you back to the hotel. It's the least I can do as your designated undercover angel."

Mitsuha looks at you, a mix of surprise and gratitude in her eyes. "Are you sure? I wouldn't want to trouble you," she says, her voice reflecting a hint of concern but also relief at the offer of help.

With a reassuring smile, you respond, "Absolutely sure. It's no trouble at all. Let's make sure you get there safely." You carefully lift her up, making sure not to cause any further discomfort to her ankle.

As you carry her back towards the hotel, Mitsuha laughs softly, "Well, this is certainly a first for me. Being carried by a charming stranger who claims to be an angel. I'll have quite the story to tell."

possible actions:
1. Ask her more about herself and why she's in Hokkaido.
2. Share a bit about why you decided to visit Hokkaido yourself.
3. Make light 

I select option 3. TURN: 4 


As you make your way back to the hotel with Mitsuha in your arms, you decide to keep the mood light and engaging. "Isn't it just breathtaking here? The colors of autumn really make this place look like a painting," you comment, gesturing towards the vibrant hues of red, orange, and yellow that adorn the trees lining the path.

Mitsuha nods, her gaze following your gesture. "It truly is beautiful. I've always loved the countryside, especially during autumn. The colors, the air, everything feels so refreshing and... pure," she says, her eyes reflecting the admiration she has for the scenery. "It's moments like these that make me appreciate the simple pleasures in life."

The conversation flows easily between you two, with occasional laughs and shared observations about the natural beauty surrounding you. The atmosphere is light and comfortable, making the walk back to the hotel seem shorter than it is.

possible actions:
1. Upon reaching the hotel, offer to 

I select option 1. TURN: 5 


Upon reaching the hotel, you gently set Mitsuha down, ensuring she's steady on her feet before letting go. "Thank you for the company on the way back. It made the walk back much more enjoyable," you say with a warm smile. "How about we have dinner together later? Consider it a thank you for letting me be your undercover angel today."

Mitsuha smiles brightly, her eyes lighting up at the suggestion. "I'd love that. It's the least I can do after you've gone out of your way to help me. Plus, it'll give me a chance to properly thank you," she replies, her tone sincere and enthusiastic. "Let's meet in the hotel's dining area? Say, around 7 PM?"

You nod in agreement, pleased with the plan. "7 PM it is. I'm looking forward to it," you respond, already anticipating the evening ahead.

With plans for dinner set, you both head to your respective rooms to freshen up, the promise of a pleasant evening lingering in the air.

**Summary of the Round:**
During this round